In [1]:
with open("than_dieu_dai_hiep.txt", encoding="utf-8") as f:
    text = f.read()

In [2]:
print("length of dataset in charactets: ", len(text))

length of dataset in charactets:  3815622


In [3]:
# Let's look at the first 1000 characters
print(text[:1000])


Kim Dung
Thần Điêu Đại Hiệp
Hồi 1
Người dị khách trên bờ hồ
Giang Nam, một dải đất hữu tình gồm nhiều thắng cảnh mà xưa nay khách giang hồ đã từng lưu gót, không một ai không quyến luyến. Bốn mùa đều có hoa nở: Xuân có thuỳ liễu xinh tươi, hạ có anh đào trắng xoá. Nhất là vào đầu thu, trong hồ sen không ngớt những tiếng hát véo von cũ những thân nhân quý tộc. Họ tiêu khiển bằng thú chèo ghe hái sen trông rất thanh lịch.Năm đó, vào thời Nam-Tống, trấn Lăng hồ, phía Bắc trai thanh gái lịch du ngoạn rất nhiềụMột hôm, vào cận tiết Trung thu, dưới hồ sen có một chiếc ghe nhỏ chở năm người thiểu nữ lơ lửng trên dòng nước như mặt gương. Năm thiếu nữ ấy có ba người tuổi suýt soát nhau. Họ vừa hái sen vừa cất tiếng hát:Sen xa hồ, sen khô nhuỵ úaHồ không sen như dải lục đìa hiuTình đời mấy kẻ biết yêuGiữa lúc đó thì hai cô bé kia tuổi mới lên chín, chưa hiểu gì tình tứ của câu hát đó, và cũng không cần để ý, đưa tay chỉ một ông lão ngồi ở hồ sen cười nức nở.Một cô bé nói:-Kìa, đến hôm nay ông l

In [4]:
# here are all the unique characters that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)

	
 !#&()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXY_abcdefghijklmnopqrstuvwxyz|¡ÀÁÂÃÈÊÓÔÕÙÚÛÝàáâãèéêìíïòóôõøùúûýĂăĐđĩũƠơƯưẠạẢảẤấẦầẩẫẬậẮắằẳẵặẹẻẽẾếỀềểỄễỆệỉịỌọỏỐốỒồổỗỘộỚớỜờỞởỡỢợụỦủứừỬửữựỳỵỷỹ“”
191


In [5]:
# Create mapping from characters to intergers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: "".join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hello world"))
print(decode(encode("hello world")))

[61, 58, 65, 65, 68, 2, 76, 68, 71, 65, 57]
hello world


In [6]:
decode([0, 1])

'\t\n'

In [7]:
# Encode the entire text dataset and store it into a torch.tensor

import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10])

torch.Size([3815622]) torch.int64
tensor([ 1, 38, 62, 66,  2, 31, 74, 67, 60,  1])


In [8]:
# Split up the data into train and validation sets
n= int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
len(train_data), len(val_data)

(3434059, 381563)

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([ 1, 38, 62, 66,  2, 31, 74, 67, 60])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
print(f"{x=}")
print(f"{y=}")

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

x=tensor([ 1, 38, 62, 66,  2, 31, 74, 67])
y=tensor([38, 62, 66,  2, 31, 74, 67, 60])
when input is tensor([1]) the target: 38
when input is tensor([ 1, 38]) the target: 62
when input is tensor([ 1, 38, 62]) the target: 66
when input is tensor([ 1, 38, 62, 66]) the target: 2
when input is tensor([ 1, 38, 62, 66,  2]) the target: 31
when input is tensor([ 1, 38, 62, 66,  2, 31]) the target: 74
when input is tensor([ 1, 38, 62, 66,  2, 31, 74]) the target: 67
when input is tensor([ 1, 38, 62, 66,  2, 31, 74, 67]) the target: 60


In [11]:
# Build dataset with block_size and batch_size
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  # generate small batch of data of input x and target y
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,)) # generate random integer from 0 to len(data) - block_size, output shape will be (batch_size, )
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs: ', xb.shape)
print(xb)
print('target: ', yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b, t]
    print(f"when input is {context.tolist()} the target: {target}")

inputs:  torch.Size([4, 8])
tensor([[  2,  60,  97,  67,   2,  56, 160,  73],
        [  2,  57, 129,  74,   2,  61,  62, 153],
        [  2, 117, 160,  73,   2,  67,  61,  95],
        [ 56, 106,   2,  73,  61, 135,  73,   2]])
target:  torch.Size([4, 8])
tensor([[ 60,  97,  67,   2,  56, 160,  73,   2],
        [ 57, 129,  74,   2,  61,  62, 153,  74],
        [117, 160,  73,   2,  67,  61,  95,   2],
        [106,   2,  73,  61, 135,  73,   2,  56]])
----
when input is [2] the target: 60
when input is [2, 60] the target: 97
when input is [2, 60, 97] the target: 67
when input is [2, 60, 97, 67] the target: 2
when input is [2, 60, 97, 67, 2] the target: 56
when input is [2, 60, 97, 67, 2, 56] the target: 160
when input is [2, 60, 97, 67, 2, 56, 160] the target: 73
when input is [2, 60, 97, 67, 2, 56, 160, 73] the target: 2
when input is [2] the target: 57
when input is [2, 57] the target: 129
when input is [2, 57, 129] the target: 74
when input is [2, 57, 129, 74] the target: 2
when i

In [12]:
print(xb) # input for transformer

tensor([[  2,  60,  97,  67,   2,  56, 160,  73],
        [  2,  57, 129,  74,   2,  61,  62, 153],
        [  2, 117, 160,  73,   2,  67,  61,  95],
        [ 56, 106,   2,  73,  61, 135,  73,   2]])


## Using Bigram LM using pytorch

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# Start with torch.zeros((1, 1) = [[1]] => start with \t
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 191])
tensor(5.8384, grad_fn=<NllLossBackward0>)
	|ÚéY/ỉĩHWôỬIFâôỳôÚịYó(kÛqøẳẻỜzặÊYĂ;ẵdịỷék_ÊợƠịiềẠú7ậIwj“ƯUnFờĂ&ỵR+ữq	ữ;7ÔãỀỳSÙ
/ẾrôgẾÙ”Vacẵ(PẢAấy8Ề“


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(100_000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.2647323608398438


## The mathematical trick in self-attention

In [16]:
torch.manual_seed(337)
B, T, C = 4, 8, 2 # Batch, Time, Channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [17]:
x[0]

tensor([[-1.3601, -0.0852],
        [ 0.6640,  0.6716],
        [ 0.8421,  1.6018],
        [-1.4630,  0.0556],
        [-1.0694,  1.0708],
        [-0.5575, -0.4001],
        [-0.3976, -0.5358],
        [ 0.6464, -1.2248]])

In [18]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bow = bag of words
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [19]:
xbow.shape

torch.Size([4, 8, 2])

Cách này rất kém hiệu quả

=> Trick: Sử dụng phép nhâ ma trận

version 2

In [20]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [21]:
wei = torch.tril(torch.ones(T, T)) # wei is short for weight
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) = (B, T, C)

In [22]:
torch.allclose(xbow, xbow2)

True

Version 3: Using softmax

In [23]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [24]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

### Position embedding

In [26]:
torch.arange(T).shape

torch.Size([8])

In [28]:
n_embed = 32
position_embedding_tablee = nn.Embedding(block_size, n_embed)
pos_emb = position_embedding_tablee(torch.arange(T))

In [30]:
pos_emb.shape

torch.Size([8, 32])

In [31]:
pos_emb

tensor([[-4.3067e-02, -1.6047e+00,  1.7878e+00, -4.7805e-01, -2.4286e-01,
         -9.3416e-01, -2.4826e-01, -1.2082e+00, -7.6884e-01,  7.6245e-01,
         -1.5673e+00, -2.3945e-01,  2.3228e+00, -9.6337e-01,  2.0024e+00,
          4.6643e-01,  8.0080e-01,  1.6806e+00,  3.5586e-01, -6.8662e-01,
         -4.9336e-01,  2.4149e-01, -1.1109e+00,  9.1546e-02, -2.5158e-01,
          8.5986e-01, -3.0973e-01, -3.9571e-01,  8.0341e-01, -6.2160e-01,
         -5.9200e-01, -6.3074e-02],
        [ 3.0572e-01, -7.7459e-01,  3.4912e-02,  3.2110e-01,  1.5736e+00,
         -8.4547e-01,  1.3123e+00,  6.8716e-01, -1.2347e+00, -4.8791e-01,
         -1.4181e+00,  8.9627e-01,  4.9905e-02,  2.2667e+00,  1.1790e+00,
         -4.3445e-01, -8.1401e-01, -7.3599e-01, -8.3712e-01, -9.2239e-01,
          1.8113e+00,  1.6056e-01,  3.6725e-01,  1.7541e-01, -1.1845e+00,
          1.3835e+00, -1.2024e+00,  7.0781e-01, -1.0759e+00,  5.3565e-01,
          1.1754e+00,  5.6117e-01],
        [-1.0546e+00,  1.2780e+00,  1.45

## Self attention

Note:

- KEY: là những gì mình có
- QUERY: là những gì mình đang tìm kiếm


In [47]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C) # ([4, 8, 32])

# let's see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)

wei = q @ k.transpose(-2, -1) # Chỉ transpose 2 chiều cuối, giữ nguyên batch, output: (B, T, T) 


tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

# out = wei @ x
v = value(x)
out = wei @ v
out.shape # [4, 8, 32]

torch.Size([4, 8, 16])

**Note**

- Attention là một **communication mechanism**. Có thể được xem như các nút trong một đồ thị có hướng nhìn vào nhau và tổng hợp thông tin bằng tổng trọng số từ tất cả các nút trỏ đến chúng, với trọng số phụ thuộc vào dữ liệu.
- Không có khái niệm về không gian. Attention chỉ đơn giản là hoạt động trên một tập hợp các vector. Đây là lý do tại sao chúng ta cần mã hóa vị trí (positionally encode) các token.
- Mỗi ví dụ trên chiều batch được xử lý hoàn toàn độc lập và không bao giờ "nói chuyện" với nhau.
- Trong một khối attention "encoder", chỉ cần xóa dòng duy nhất thực hiện masking với `tril`, cho phép tất cả các token giao tiếp. Khối này được gọi là khối attention "decoder" vì nó có masking hình tam giác và thường được sử dụng trong các thiết lập tự hồi quy (autoregressive), như mô hình hóa ngôn ngữ.
- "self-attention" chỉ có nghĩa là các key và value được tạo ra từ cùng một nguồn với query. Trong "cross-attention", các query vẫn được tạo ra từ x, nhưng các key và value đến từ một nguồn bên ngoài khác (ví dụ: một mô-đun encoder).
- Attention "Scaled" chia thêm `wei` cho 1/sqrt(head_size). Điều này làm cho khi đầu vào Q, K có phương sai đơn vị, wei cũng sẽ có phương sai đơn vị và Softmax sẽ vẫn khuếch tán và không bị bão hòa quá nhiều. Minh họa bên dưới.


In [37]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)

In [ ]:
k.var() # Phương sai

tensor(1.0449)

In [39]:
q.var()

tensor(1.0700)

In [40]:
wei.var()

tensor(17.4690)

In [42]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [43]:
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [44]:
wei.var()

tensor(1.0918)

In [46]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])